In [270]:
import simplejson as json  
import os
import lzma
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
pd.set_option('display.max_columns', None)
nlp = spacy.load('en')


In [271]:
%cd ""
cases_df = pd.read_csv('nhcase_decisions.csv')

C:\Users\Nick\Downloads


In [272]:
# #Loading data
# cases = []
 
# with lzma.open('data.jsonl (6).xz') as infile:
#     for line in infile:
#         #decode the file into a convenient format
#         record = json.loads(str(line, 'utf-8'))
#         #if the decision date on the case matches one we're interested in, add to our list
# #         if (int(record['decision_date'].split("-")[0])>2000):
# #             cases.append(record)
#         cases.append(record)

# print("Number of Cases: {}".format(len(cases)))

In [273]:
# #Extracting meta data and text of cases
# def parse_cases(case):
    
#     elem_dict = {}
#     try:
#         elem_dict["id"] = case['id']
#     except IndexError:
#         elem_dict["id"] = np.nan
#     try:
#         elem_dict["name"] = case['name']
#     except IndexError:
#         elem_dict["name"] = np.nan
#     try:   
#         elem_dict["name_abbreviation"] = case['name_abbreviation']
#     except IndexError:
#         elem_dict["name_abbreviation"] = np.nan
#     try:
#         elem_dict["parties"] = case['casebody']['data']['parties']
#     except IndexError:
#          elem_dict["parties"] = np.nan
#     try:
#         elem_dict["plaintiff"] = case['name_abbreviation'].split('v.')[0].strip()
#     except IndexError:
#         elem_dict["plaintiff"] = np.nan
#     try:
#         elem_dict["defendant"] = case['name_abbreviation'].split('v.')[1].strip()
#     except IndexError:
#         elem_dict["defendant"] = np.nan
#     try:
#         elem_dict["decision_date"] = case['decision_date']
#     except IndexError:
#         elem_dict["decision_date"] = np.nan
#     try:
#         elem_dict["docket_number"] = case['docket_number']
#     except IndexError:
#         elem_dict["docket_number"] = np.nan
        
#     citation = [citation for citation in case['citations'] if citation['type'] == 'official'][0]
#     elem_dict["citation_type"] = citation['type']
#     elem_dict["citation_num"] = citation['cite']
#     try:
#         elem_dict["reporter"] = case['reporter']['full_name']
#     except IndexError:
#         elem_dict["reporter"] = np.nan
#     try:
#         elem_dict["court_id"] = case['court']['id']
#     except IndexError:
#         elem_dict["court_id"] = np.nan
#     elem_dict["court_name"] = case['court']['name']
#     elem_dict["jurisdiction_id"] = case['jurisdiction']['id']
#     elem_dict["jurisdiction_name"] = case['jurisdiction']['name_long']
#     try:
#         elem_dict["plaintiff_attorneys"] = case['casebody']['data']['attorneys'][0]
#     except IndexError:
#         elem_dict["plaintiff_attorneys"] = np.nan
#     try:
#         elem_dict["defendants_attorneys"] = case['casebody']['data']['attorneys'][1]
#     except IndexError:
#         elem_dict["defendants_attorneys"] = np.nan
#     elem_dict["judges"] = case['casebody']['data']['judges']
#     opinion_type = [opinion['type'] for opinion in case['casebody']['data']['opinions']]
#     elem_dict["opinion_type"]= opinion_type
#     opinion_author = [opinion['author'] for opinion in case['casebody']['data']['opinions']]
#     elem_dict["opinion_author"] = opinion_author
#     elem_dict["head_matter"] = case['casebody']['data']['head_matter']
#     opinion_text = [opinion['text'] for opinion in case['casebody']['data']['opinions']]
#     elem_dict["opinion_text"] = opinion_text
    
#     return elem_dict

In [274]:
# #parsing the cases
# parsed_cases = []
# for case in cases:
#     doc = parse_cases(case)
#     parsed_cases.append(doc)
    
# cases_df = pd.DataFrame(parsed_cases)
# cases_df.head()

In [275]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

### Defendant Attorneys Into a Clean List

In [276]:
list_of_defendant_attorneys = []

for i in range(len(cases_df)):
            try:
                doc = nlp(cases_df['defendants_attorneys'][i])
                NER = pd.DataFrame([(X.text, X.label_) for X in doc.ents])
                NER = NER.loc[NER[1] == 'PERSON']
                NER = NER.loc[NER[0] != 'Nashua']
                names = list(NER[0])
                list_of_defendant_attorneys.append(names)
            except:
                list_of_defendant_attorneys.append(['NA'])


df = pd.DataFrame({'Defendant_Attorneys': list_of_defendant_attorneys})
cases_df = cases_df.merge(df, how='outer', left_index=True, right_index=True)

### Plantiff Attorneys Into a Clean List

In [277]:
list_of_plaintiff_attorneys = []

for i in range(len(cases_df)):
            try:
                doc = nlp(cases_df['plaintiff_attorneys'][i])
                NER = pd.DataFrame([(X.text, X.label_) for X in doc.ents])
                NER = NER.loc[NER[1] == 'PERSON']
                NER = NER.loc[NER[0] != 'Nashua']
                names = list(NER[0])
                list_of_plaintiff_attorneys.append(names)
            except:
                list_of_plaintiff_attorneys.append(['NA'])


df2 = pd.DataFrame({'Plaintiff_Attorneys': list_of_plaintiff_attorneys})
cases_df = cases_df.merge(df2, how='outer', left_index=True, right_index=True)

### To get Plaintiff_Attorneys all in their own rows

In [278]:
# pd.DataFrame({
# col:np.repeat(cases_df[col].values, cases_df[lst_col].str.len())
# for col in cases_df.columns.difference([lst_col])
# }).assign(**{lst_col:np.concatenate(cases_df[lst_col].values)})[cases_df.columns.tolist()]


cases_df.columns
cases_df['index1'] = cases_df.index
res = cases_df.set_index(['citation_num', 'citation_type', 'court_id', 'court_name',
       'decision_date', 'defendant', 'defendants_attorneys', 'docket_number',
       'head_matter', 'id'])['Plaintiff_Attorneys'].apply(pd.Series).stack()
res = res.reset_index()
res.head()

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,level_10,0
0,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,0,Wilton
1,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,1,John P Griffith
2,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,0,Philip T. McLaughlin
3,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,1,John C. Kissinger
4,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,2,Patrick E. Donovan


### Merge this new DF onto the old one

In [279]:
res.columns
cases_df_maybe = res.merge(cases_df, how='left', on = ['citation_num', 'citation_type', 'court_id', 'court_name',
                           'decision_date', 'defendant', 'defendants_attorneys', 'docket_number', 'head_matter', 'id'])
cases_df_maybe.head()

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,level_10,0,Unnamed: 0,judges,jurisdiction_id,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter,decision,Defendant_Attorneys,Plaintiff_Attorneys,index1
0,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,0,Wilton,0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0
1,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,1,John P Griffith,0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0
2,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,0,Philip T. McLaughlin,1,"['THAYER, J., with whom HORTON, J., joined, co...",47.0,New Hampshire,The State of New Hampshire v. Eric Bennett,State v. Bennett,"['BRODERICK, j.', 'THAYER, J.,']","['BRODERICK, j.\nAfter a jury trial in Superio...","['majority', 'concurrence']",['The State of New Hampshire v. Eric Bennett'],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports,['Affirmed'],[Janice S. Peterson],"[Philip T. McLaughlin, John C. Kissinger, Patr...",1
3,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,1,John C. Kissinger,1,"['THAYER, J., with whom HORTON, J., joined, co...",47.0,New Hampshire,The State of New Hampshire v. Eric Bennett,State v. Bennett,"['BRODERICK, j.', 'THAYER, J.,']","['BRODERICK, j.\nAfter a jury trial in Superio...","['majority', 'concurrence']",['The State of New Hampshire v. Eric Bennett'],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports,['Affirmed'],[Janice S. Peterson],"[Philip T. McLaughlin, John C. Kissinger, Patr...",1
4,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,2,Patrick E. Donovan,1,"['THAYER, J., with whom HORTON, J., joined, co...",47.0,New Hampshire,The State of New Hampshire v. Eric Bennett,State v. Bennett,"['BRODERICK, j.', 'THAYER, J.,']","['BRODERICK, j.\nAfter a jury trial in Superio...","['majority', 'concurrence']",['The State of New Hampshire v. Eric Bennett'],State,"Philip T. McLaughlin, attorney general (John C...",New Hampshire Reports,['Affirmed'],[Janice S. Peterson],"[Philip T. McLaughlin, John C. Kissinger, Patr...",1


### To get all the Plaintiff_Attorneys in their own rows

In [280]:
cases_df_maybe.columns
cases_df['index1'] = cases_df.index
res2 = cases_df.set_index(['citation_num', 'citation_type', 'court_id', 'court_name',
       'decision_date', 'defendant', 'defendants_attorneys', 'docket_number',
       'head_matter', 'id'])['Defendant_Attorneys'].apply(pd.Series).stack()
res2 = res2.reset_index()
res2.head()

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,level_10,0
0,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,0,Stephen L. Tober
1,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,1,Tara C. Schoff
2,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,2,Tober
3,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,3,Steven E. Feld
4,144 N.H. 13,official,8797.0,New Hampshire Supreme Court,1999-07-13,Bennett,"Janice S. Peterson, public defender, of Keene,...",No. 97-387,No. 97-387\r\nThe State of New Hampshire v. Er...,105095.0,0,Janice S. Peterson


### Merge this DF to the old one...should be more than 33k rows

In [283]:
cases_df_maybe.columns
cases_df_maybe2 = res2.merge(cases_df_maybe, how='left', on = ['citation_num', 'citation_type', 'court_id', 'court_name',
                           'decision_date', 'defendant', 'defendants_attorneys', 'docket_number', 'head_matter', 'id'])
cases_df_maybe2.columns


cases_df_maybe2.columns = ['citation_num', 'citation_type', 'court_id', 'court_name',
       'decision_date', 'defendant', 'defendants_attorneys', 'docket_number',
       'head_matter', 'id', 'Plaintiff_Attorney_Clean_number', 'Plaintiff_Attorneys_Clean',
        'Defendant_Attorney_Clean_number', 'Defendant_Attorneys_Clean',
       'Unnamed: 0', 'judges', 'jurisdiction_id', 'jurisdiction_name', 'name',
       'name_abbreviation', 'opinion_author', 'opinion_text', 'opinion_type',
       'parties', 'plaintiff', 'plaintiff_attorneys', 'reporter', 'decision',
       'Defendant_Attorneys', 'Plaintiff_Attorneys', 'index1']



cases_df_maybe2.head()

,citation_num,citation_type,court_id,court_name,decision_date,defendant,defendants_attorneys,docket_number,head_matter,id,Plaintiff_Attorney_Clean_number,Plaintiff_Attorneys_Clean,Defendant_Attorney_Clean_number,Defendant_Attorneys_Clean,Unnamed: 0,judges,jurisdiction_id,jurisdiction_name,name,name_abbreviation,opinion_author,opinion_text,opinion_type,parties,plaintiff,plaintiff_attorneys,reporter,decision,Defendant_Attorneys,Plaintiff_Attorneys,index1
0,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,0,Stephen L. Tober,0.0,Wilton,0.0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0.0
1,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,0,Stephen L. Tober,1.0,John P Griffith,0.0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0.0
2,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,1,Tara C. Schoff,0.0,Wilton,0.0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0.0
3,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,1,Tara C. Schoff,1.0,John P Griffith,0.0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0.0
4,144 N.H. 131,official,8797.0,New Hampshire Supreme Court,1999-08-30,NaN,"Tober Law Offices, P.A., of Portsmouth (Stephe...",No. LD-97-009,"No. LD-97-009\r\nFeld’s Case\r\nAugust 30, 199...",105092.0,2,Tober,0.0,Wilton,0.0,"['THAYER, HORTON, and BRODERICK, JJ., did not ...",47.0,New Hampshire,Feld’s Case,Feld’s Case,"['JOHNSON, J.']","[""JOHNSON, J.\nThe Supreme Court Committee on ...",['majority'],['Feld’s Case'],Feld’s Case,"Griffith & Associates, PLLC, of Wilton (John P...",New Hampshire Reports,['So ordered'],"[Stephen L. Tober, Tara C. Schoff, Tober, Stev...","[Wilton, John P Griffith]",0.0


In [284]:
cases_df_maybe2.to_csv('NH_Cases_Attorneys_Own_Rows.csv')